# Prospects of Restaurants in Seoul

## Week 1


---
### Part 1 (Problem)
A description of the problem and a discussion of the background.
Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

Where in Seoul would it be best to open a restaurant. What is the place that has the most reviews, what location seems to be the most popular. Is there a type of restaurant that is more popular than another?

The problem will be solved using the Foursquare API, looking at the usercount and tipcount. This problem will help everyone that wants to open up a restaurant in Rio de Janeiro and wants to know if it is a viable option where they intend on opening.

---
### Part 2 (Data)

The data that will be used is Foursquare data for Seoul city. It will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant.

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [2]:
address = "Seoul"

geolocator = Nominatim(user_agent="rio_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Seoul is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seoul is 37.5666791, 126.9782914.


In [3]:
CLIENT_ID = 'LH1TXEWZCYCZA3SEHWID1KLQSDF5U1K0ZPOXWTAKKMVK2NGT'
CLIENT_SECRET = 'PCKXM005IPX5EF1TSZHWMB5NAT3LWPWNTL33X0JIRYZQCWFH'
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LH1TXEWZCYCZA3SEHWID1KLQSDF5U1K0ZPOXWTAKKMVK2NGT
CLIENT_SECRET:PCKXM005IPX5EF1TSZHWMB5NAT3LWPWNTL33X0JIRYZQCWFH


In [4]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/search?client_id=LH1TXEWZCYCZA3SEHWID1KLQSDF5U1K0ZPOXWTAKKMVK2NGT&client_secret=PCKXM005IPX5EF1TSZHWMB5NAT3LWPWNTL33X0JIRYZQCWFH&ll=37.5666791,126.9782914&v=20180605&query=Restaurant&radius=50000&limit=100'

In [6]:
results = requests.get(url).json()

venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,50725938e4b0c9f5d0bc1ef2,Crystal Jade Restaurant (크리스탈제이드),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1596015936,False,중구 남대문로7길 16,소공점,37.564009,126.981100,"[{'label': 'display', 'lat': 37.56400858535728...",387,04533,KR,서울특별시,서울특별시,대한민국,"[중구 남대문로7길 16 (소공점), 소공동, 서울특별시, 서울특별시, 04533,...",NaN
1,4cd2945b773ba14371a2bbe3,New Maul Restaurant (새마을식당),"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",v-1596015936,False,중구 세종대로20길 19,무교점,37.567012,126.978286,"[{'label': 'display', 'lat': 37.56701218666752...",37,100-101,KR,서울특별시,서울특별시,대한민국,"[중구 세종대로20길 19 (무교점), 명동, 서울특별시, 서울특별시, 100-10...",NaN
2,58b2224f1174203d153d44fd,NOOK Restaurant (누크 레스토랑),"[{'id': '52e81612bcbc57f1066b79f4', 'name': 'B...",v-1596015936,False,중구 남대문로 56,NaN,37.562095,126.981614,"[{'label': 'display', 'lat': 37.562095, 'lng':...",588,04535,KR,서울특별시,서울특별시,대한민국,"[중구 남대문로 56, 명동, 서울특별시, 서울특별시, 04535, 대한민국]",NaN
3,4be7da87ee96c928325dfdbf,New Maul Restaurant (새마을식당),"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",v-1596015936,False,중구 태평로2가 56-1,북창2호점,37.564148,126.977110,"[{'label': 'display', 'lat': 37.56414813002853...",300,NaN,KR,서울특별시,서울특별시,대한민국,"[중구 태평로2가 56-1 (북창2호점), 소공동, 서울특별시, 서울특별시, 대한민국]",NaN
4,4b058784f964a520349722e3,The Restaurant (더 레스토랑),"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",v-1596015936,False,종로구 삼청로 54,NaN,37.580799,126.980352,"[{'label': 'display', 'lat': 37.58079893639559...",1582,03053,KR,서울특별시,서울특별시,대한민국,"[종로구 삼청로 54, 삼청동, 서울특별시, 서울특별시, 03053, 대한민국]",NaN


In [7]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Crystal Jade Restaurant (크리스탈제이드),Chinese Restaurant,중구 남대문로7길 16,소공점,37.564009,126.981100,"[{'label': 'display', 'lat': 37.56400858535728...",387,04533,KR,서울특별시,서울특별시,대한민국,"[중구 남대문로7길 16 (소공점), 소공동, 서울특별시, 서울특별시, 04533,...",NaN,50725938e4b0c9f5d0bc1ef2
1,New Maul Restaurant (새마을식당),BBQ Joint,중구 세종대로20길 19,무교점,37.567012,126.978286,"[{'label': 'display', 'lat': 37.56701218666752...",37,100-101,KR,서울특별시,서울특별시,대한민국,"[중구 세종대로20길 19 (무교점), 명동, 서울특별시, 서울특별시, 100-10...",NaN,4cd2945b773ba14371a2bbe3
2,NOOK Restaurant (누크 레스토랑),Buffet,중구 남대문로 56,NaN,37.562095,126.981614,"[{'label': 'display', 'lat': 37.562095, 'lng':...",588,04535,KR,서울특별시,서울특별시,대한민국,"[중구 남대문로 56, 명동, 서울특별시, 서울특별시, 04535, 대한민국]",NaN,58b2224f1174203d153d44fd
3,New Maul Restaurant (새마을식당),BBQ Joint,중구 태평로2가 56-1,북창2호점,37.564148,126.977110,"[{'label': 'display', 'lat': 37.56414813002853...",300,NaN,KR,서울특별시,서울특별시,대한민국,"[중구 태평로2가 56-1 (북창2호점), 소공동, 서울특별시, 서울특별시, 대한민국]",NaN,4be7da87ee96c928325dfdbf
4,The Restaurant (더 레스토랑),French Restaurant,종로구 삼청로 54,NaN,37.580799,126.980352,"[{'label': 'display', 'lat': 37.58079893639559...",1582,03053,KR,서울특별시,서울특별시,대한민국,"[종로구 삼청로 54, 삼청동, 서울특별시, 서울특별시, 03053, 대한민국]",NaN,4b058784f964a520349722e3


In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map